In [34]:
f_in = open('shared_data/new york_anon_locationData_newcrawl.txt', 'r')
foursquare_venues = {}
for line in f_in:
	splits = line.split('*;*')
	venueid = splits[0]
	info = eval(splits[1])
	foursquare_venues[venueid] = info

In [37]:
# ny_venues = pd.DataFrame.from_dict(foursquare_venues, orient='index', 
#                        columns=['latitude', 'longitude', 'category', 'unique_users', 'total_check-ins', 'title'])
# ny_venues.to_csv('shared_data/ny_venues.csv', 
#                  columns=['latitude', 'longitude', 'category', 'unique_users', 'total_check-ins', 'title'])

In [1]:
import pandas as pd
transitions = pd.read_csv('shared_data/new york_placenet_transitions.txt', sep=',', names=['A_id', 'B_id','A_datetime', 'B_datetime'])
ny_venues = pd.read_csv('shared_data/ny_venues.csv', sep=',')

In [2]:
transitions.head()

,A_id,B_id,A_datetime,B_datetime
0,34814,32616,2011-04-01 21:46:59,2011-04-03 23:25:17
1,6136,41982,2011-04-06 03:03:56,2011-04-06 03:22:48
2,41982,35981,2011-04-06 03:22:48,2011-04-07 18:00:53
3,35981,6701,2011-04-07 18:00:53,2011-04-07 18:01:06
4,6701,57814,2011-04-07 18:01:06,2011-04-08 22:38:26


In [2]:
ny_venues.head()

,venue_id,latitude,longitude,category,unique_users,total_check-ins,title
0,42889,40.760265,-73.989105,Italian,217,291,Ristorante Da Rosina
1,57489,40.780704,-73.954704,Bakery,1291,2231,Le Pain Quotidien
2,42890,40.663925,-74.118230,Video Store,60,186,Blockbuster
3,74771,40.739840,-73.991770,Corporate ' Office,6,12,Day & Night Office
4,42891,40.786987,-73.807530,Other - Food,29,40,Erin's Isle


In [3]:
ny_venues[ny_venues.title == 'Blockbuster'].head()
    
    
    
    
    

,venue_id,latitude,longitude,category,unique_users,total_check-ins,title
2,42890,40.663925,-74.118230,Video Store,60,186,Blockbuster
3331,1685,40.661945,-73.695305,Video Store,73,230,Blockbuster
13754,6846,40.662220,-73.840780,Video Store,26,51,Blockbuster
19878,52811,40.730320,-73.888030,Video Store,27,58,Blockbuster
21738,53737,40.775980,-73.910510,Video Store,33,104,Blockbuster


In [17]:
venues

40.763992

In [4]:
print(max(ny_venues.latitude))
print(min(ny_venues.latitude))
print(max(ny_venues.longitude))
print(min(ny_venues.longitude))
# for index, row in ny_venues.iterrows():
#     print(row.latitude

40.99928
40.415005
-73.48129
-74.28631


In [10]:
from geopy import distance
from math import pi, cos


class Geometry:
    R_EARTH = 6371.0 
    
    #calculating new latitude in n meters 
    @staticmethod
    def _get_latitude(latitude, n_meters):
        if abs(latitude)>90:
            return 0
        dy = n_meters/1000
        return latitude  + (dy / Geometry.R_EARTH) * (180 / pi);
    
    #calculating new longtitude in n meters 
    @staticmethod
    def _get_longtitude(latitude, longitude, n_meters):
#         check real values
        if abs(latitude)>90 or abs(longitude)>180:
            return 0
        dx = n_meters/1000
        return longitude + (dx / Geometry.R_EARTH) * (180 / pi) / cos(latitude * pi/180);
    
    @staticmethod
    def get_places_around(center_coordinates, radius, venues):
#         circle area
        places_ids = []
        for index, row in venues.iterrows():
            dist = distance.distance(center_coordinates, (row.latitude, row.longitude)).m
            if dist<=radius:
                places_ids.append(row.venue_id)
        return places_ids
    
    def get_rect_points(center, size):
        radius = size/2
        
        # A-B
        # D-C
        a_lat = Geometry._get_latitude(center[0], radius)
        a_long = Geometry._get_longtitude(center[0], center[1], -radius)
        
        b_lat = Geometry._get_latitude(center[0], radius)
        b_long = Geometry._get_longtitude(center[0], center[1], radius)
        
        c_lat = Geometry._get_latitude(center[0], -radius)
        c_long = Geometry._get_longtitude(center[0], center[1], radius)
        
        d_lat = Geometry._get_latitude(center[0], -radius)
        d_long = Geometry._get_longtitude(center[0], center[1], -radius)
        
        coordinates = [(a_lat, a_long), (b_lat, b_long), (c_lat, c_long), (d_lat, d_long)]
        return coordinates
    
    def get_cell_centres:
        

In [12]:
Geometry.get_rect_points(center=(40, -73), size=200)

[(40.00089932160592, -73.00117398097981),
 (40.00089932160592, -72.99882601902019),
 (39.99910067839408, -72.99882601902019),
 (39.99910067839408, -73.00117398097981)]

In [23]:
distance.distance((40, 74), (40, 74.0025)).m

213.48464238941

In [ ]:
# from geometry import Geometry 
places = Geometry.get_places_around(center_coordinates=(40.760265,-73.989105), radius=500, venues=ny_venues)
print(places)

In [52]:
import numpy as np
lonMin = -74.1  # minimum longitude
lonMax = -73.7
lonStep = 0.01 # defines cell size
 
latMin = 40.6  # minimum latitude
latMax = 41.0
latStep = 0.01  # defines cell size
 
latLen = int((latMax - latMin) / latStep) + 1  # number of cells on the y-axis
lonLen = int((lonMax - lonMin) / lonStep) + 1  # number of cells on the x-axis

venues_grid=np.zeros((latLen, lonLen), dtype=np.ndarray)

for index, row in ny_venues.iterrows():
        lat = row.latitude
        longit = row.longitude
        # if outside the grid then ingore point
        if (lat < latMin) or (lat > latMax) or (longit < lonMin) or (longit > lonMax):
            continue
        cx = int((longit - lonMin) / lonStep)
        cy = int((lat - latMin) / latStep)        
        venues_grid[cy, cx]=np.append(venues_grid[cy, cx], row.venue_id)
            



In [12]:
import numpy as np
a = np.zeros((10, 10))

In [30]:
a = np.array([])

In [1]:
from feature_extractor import FeatureExtractor
import pandas as pd
transitions = pd.read_csv('shared_data/new york_placenet_transitions.txt', sep=',', names=['A_id', 'B_id','A_datetime', 'B_datetime'])
ny_venues = pd.read_csv('shared_data/ny_venues.csv', sep=',')
extractor = FeatureExtractor(ny_venues, transitions)
extractor.calculate_squares()

In [2]:
extractor.venues_grid[36]

array([array([], dtype=float64), array([], dtype=float64),
       array([], dtype=float64), array([], dtype=float64),
       array([], dtype=float64), array([], dtype=float64),
       array([], dtype=float64), array([], dtype=float64),
       array([], dtype=float64), array([], dtype=float64),
       array([], dtype=float64), array([], dtype=float64),
       array([], dtype=float64), array([], dtype=float64),
       array([], dtype=float64), array([], dtype=float64),
       array([], dtype=float64), array([], dtype=float64),
       array([], dtype=float64), array([], dtype=float64),
       array([], dtype=float64), array([], dtype=float64),
       array([51661., 64976., 54656., 81138., 72815., 73925., 75841., 81401.,
       37537., 39780., 41784.]),
       array([62647., 23337., 30373., 77661.]),
       array([  111., 45096., 14375., 78182.]),
       array([82218., 38532., 21308., 66912., 24938., 31060., 39018., 42707.]),
       array([], dtype=float64), array([], dtype=float64),
     

In [28]:
import numpy as np
a = np.zeros((10, 10))

In [35]:
b = (1, 2)
c = (1, 2)
b==c

True

In [33]:
type(b)

tuple

In [21]:
b = [1, 2, 3]
mean(b)

NameError: name 'mean' is not defined

In [23]:
c=[]
len(c)

0

In [24]:
data = pd.read_csv('features.csv')

In [25]:
data.head()

,Unnamed: 0,cy,cx,latitude,longitude,density,neighbors_entropy,competitiveness,average_check_ins
0,0,0.0,0.0,40.65,-74.05,0.0,0.000000,0.00,0.0
1,1,0.0,1.0,40.65,-74.04,0.0,0.000000,0.00,0.0
2,2,0.0,2.0,40.65,-74.03,1.0,0.000000,0.00,0.0
3,3,0.0,3.0,40.65,-74.02,8.0,2.405639,0.00,0.0
4,4,0.0,4.0,40.65,-74.01,25.0,4.665169,-0.04,0.0


In [68]:
ny_venues[ny_venues.venue_id==0]['total_check-ins'].values[0]

43

In [63]:
# transitions = pd.read_csv('shared_data/new york_placenet_transitions.txt', sep=',', names=['A_id', 'B_id','A_datetime', 'B_datetime'])
features2 = pd.read_csv('features2.csv', sep=',')
ny_venues = pd.read_csv('shared_data/ny_venues.csv', sep=',')

In [99]:
import numpy as np
import pandas as pd
filename = 'features2.csv'

def tarik_model(dens, area_pop, trans_dens, inc_flow, neighb_entr, compet):
    return (dens*0.3+area_pop*0.02+trans_dens*0.02+inc_flow*0.02+neighb_entr*0.3+compet*0.3)*6

features_frame = pd.read_csv(filename, sep=',')

predicts = np.array([])
for index, row in features_frame.iterrows():
    predicted_value = tarik_model(row.density, row.area_popularity, row.transition_density, row.incoming_flow,
                                 row.neighbors_entropy, row.competitiveness)
    predicts = np.append(predicts, predicted_value)




array([  75.25948493,  143.72634633,  923.97498733,  323.73907037,
       1177.47612794,   78.88602004,   29.13947057,   24.9       ,
         41.55741763,  113.74767001])

In [92]:
features_frame.head(10)

,Unnamed: 0,latitude,longitude,density,neighbors_entropy,competitiveness,area_popularity,transition_density,incoming_flow,average_check_ins,venue_id
0,0,40.646133,-73.90315,20.0,4.510825,-0.100000,171.0,37.0,53.0,81.0,59479.0
1,1,40.700645,-73.94293,43.0,5.007660,-0.093023,326.0,97.0,56.0,137.0,162.0
2,2,40.702366,-74.01126,126.0,7.167056,-0.047619,3802.0,670.0,1231.0,296.0,50006.0
3,3,40.745537,-73.90356,72.0,9.535595,-0.013889,983.0,286.0,206.0,165.0,14304.0
4,4,40.827675,-73.92475,57.0,9.606036,-0.052632,5837.0,1388.0,1589.0,233.0,488.0
5,5,40.746216,-73.61775,29.0,4.395682,-0.103448,107.0,23.0,28.0,171.0,817.0
6,6,40.700405,-73.81420,5.0,1.921928,-0.200000,97.0,32.0,13.0,31.0,79385.0
7,7,40.702248,-73.79154,8.0,2.750000,-0.250000,32.0,10.0,8.0,97.0,1226.0
8,8,40.870617,-73.84654,10.0,3.654121,-0.100000,97.0,11.0,35.0,74.0,1437.0
9,9,40.741860,-73.64054,27.0,7.230187,-0.037037,303.0,48.0,84.0,283.0,1512.0
